In [3]:
## We are using the 'colorability' db credentials for all association ratings experiments. 
## Don't get confused. Yes, that means you Kushin.

##make sure you're connected to UW VPN

In [4]:
import os
import mysql.connector
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from scipy import stats
from collections import Counter


In [5]:
# db = mysql.connector.connect(
#    host='127.0.0.1',
#     port= 3306,
#    user='root',
#    passwd='kushinm11',
#     db = 'db1',
#     auth_plugin='mysql_native_password'
# )
# table_name = ''

## Connect to DB

In [6]:
db = mysql.connector.connect(
   host="schlosslab-exp.wid.wisc.edu",
    port= 3306,
   user="colorability_op",
   passwd="auy2fovei7Bee6uC",
    db = 'colorability'
)


In [7]:
##get list of table names

cursor = db.cursor()

cursor.execute("SHOW TABLES")    # execute 'SHOW TABLES' (but data is not returned)
for (table_name,) in cursor:
    print(table_name)

condition_log
data_log
register_log


In [8]:
## this is for the dummy repo i made for Tim's lab

# db = mysql.connector.connect(
#    host="kclab-exp.wid.wisc.edu",
#     port= 3306,
#    user="color_concept_rate_op",
#    passwd="eeve8yai8pohZoh9",
#     db = 'color_concept_rate'
# )
# table_name = 'register_log'

In [9]:
table_name = 'data_log'
df = pd.read_sql_query("SELECT * FROM {}".format(table_name),
    db) ### convert table into dataframe



In [10]:
## How many in each category?
df[df.trial_type=='image-slider-responseMAS'].groupby('concept')['subject_id'].apply(lambda x: len(list(np.unique(x))))


concept
animals        48
automobiles    52
values         52
weather        53
Name: subject_id, dtype: int64

In [56]:
print('How many subjects so far?')
print(df.workerID.nunique())

How many subjects so far?
221


205

In [11]:
df.prompt.unique()

array([None, 'love', 'peace', 'justice', 'evil', 'greed', 'car',
       'airplane', 'boat', 'plane', 'truck', 'hurricane', 'drought',
       'blizzard', 'sandstorm', 'lightning', 'bear', 'lion', 'bird',
       'fish', 'frog'], dtype=object)

In [32]:
d_cb = df[df.trial_type=='survey-multi-choice'] ## dataframe with colorblind info
cb_exclude = []
for i, row in d_cb.iterrows():
    if row['responses'].decode('UTF-8').split('"')[3] == 'Yes':
        cb_exclude.append(row['subject_id'])

    

In [33]:
### folks to exclude
cb_exclude

['ymypo4bmnhbz4911aecn',
 'u1n0lme13vcrg58fgdaz',
 '3ug8q1arswdwou0ohz25',
 'yhs288z0nj94g7u9yr1r',
 'fgrv67037p73qz14eb8v',
 'qdtj2op8ebe1xvj7xyqx']

In [47]:
gender_responses=[]
age_responses = []

for i, row in df[df.trial_type=='survey-text'].iterrows():
    gender_responses.append(row['responses'].decode('UTF-8').split('"')[3])
    age_responses.append(row['responses'].decode('UTF-8').split('"')[7])


Counter(gender_responses)


['{', 'Gender', ':', 'female', ',', 'Age', ':', '18', ',', 'Languages', ':', 'english, spanish, korean', '}']


In [48]:
Counter(age_responses)

Counter({'18': 123,
         '19': 51,
         '24': 1,
         '21': 8,
         '20': 15,
         '18 ': 1,
         '25': 2,
         '17': 2,
         '23': 1,
         '8': 1})

In [57]:
df_final = df[~(df.subject_id.isin(cb_exclude))&(df.trial_type=='image-slider-responseMAS')]

In [58]:
df_final.subject_id.nunique()

199

In [61]:
df_final.response = df_final.response + 200

/usr/local/lib/python3.9/site-packages/pandas/core/generic.py:5491: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [69]:
df_final

,responses,question_order,trial_type,trial_index,rt,time_elapsed,condition,response,prompt,color_rgb,concept,time_finished,rowID,color_index,concept_category,subject_id,workerID
9,None,None,image-slider-responseMAS,9,8855.44,80373.0,None,210.0,love,"[100, 53, 97, 57, 101, 52]",values,1.612350e+12,106554,16.0,None,a17uet7exuj4ldasvvdh,70360
10,None,None,image-slider-responseMAS,10,5457.42,86834.0,None,90.0,love,"[56, 53, 53, 56, 102, 52]",values,1.612350e+12,106555,2.0,None,a17uet7exuj4ldasvvdh,70360
11,None,None,image-slider-responseMAS,11,2971.40,90809.0,None,47.0,love,"[54, 55, 99, 102, 53, 99]",values,1.612350e+12,106556,51.0,None,a17uet7exuj4ldasvvdh,70360
12,None,None,image-slider-responseMAS,12,2792.73,94603.0,None,241.0,love,"[101, 51, 49, 98, 55, 51]",values,1.612350e+12,106557,34.0,None,a17uet7exuj4ldasvvdh,70360
13,None,None,image-slider-responseMAS,13,2777.41,98382.0,None,0.0,love,"[56, 51, 55, 54, 52, 99]",values,1.612350e+12,106558,42.0,None,a17uet7exuj4ldasvvdh,70360
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76678,None,None,image-slider-responseMAS,367,3640.80,57310100.0,None,230.0,lightning,"[51, 98, 51, 98, 51, 98]",weather,1.632270e+12,183223,25.0,None,mrmvgw1xc5vhdosvp3w9,74976
76679,None,None,image-slider-responseMAS,368,4951.10,57316000.0,None,122.0,lightning,"[98, 97, 101, 97, 97, 102]",weather,1.632270e+12,183224,40.0,None,mrmvgw1xc5vhdosvp3w9,74976
76680,None,None,image-slider-responseMAS,369,2352.10,57319400.0,None,161.0,lightning,"[101, 100, 100, 99, 97, 100]",weather,1.632270e+12,183225,44.0,None,mrmvgw1xc5vhdosvp3w9,74976
76681,None,None,image-slider-responseMAS,370,2979.30,57323400.0,None,238.0,lightning,"[50, 102, 54, 101, 102, 54]",weather,1.632270e+12,183226,0.0,None,mrmvgw1xc5vhdosvp3w9,74976
